In [28]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
file_dir = "/Users/gaucing/DataAnalytics/movies-etl/Resources"

In [3]:
with open(f"{file_dir}/wikipedia-movies.json", mode="r") as file:
    wiki_movies_raw = json.load(file)

In [4]:
kaggle_metadata = pd.read_csv(f"{file_dir}/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv(f"{file_dir}/ratings.csv")

In [5]:
# remove non-films from data
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [6]:
# keys for alternate titles
# each key is a column name whose data is an alternate title for its film
alt_title_keys = ["Also known as","Arabic","Cantonese","Chinese","French",
                  "Hangul","Hebrew","Hepburn","Japanese","Literally",
                  "Mandarin","McCune–Reischauer","Original title","Polish",
                  "Revised Romanization","Romanized","Russian",
                  "Simplified","Traditional","Yiddish"]

# column names that can be consolidated
# each key is a current name and each value is a replacement name
column_names_to_change = {"Animation by" : "Animator(s)",
                          "Directed by" : "Director",
                          "Distributed by" : "Distributor",
                          "Edited by" : "Editor(s)",
                          "Music by" : "Composer(s)",
                          "Theme music composer" : "Composer(s)",
                          "Producer" : "Producer(s)",
                          "Produced by" : "Producer(s)",
                          "Productioncompanies " : "Production company(s)",
                          "Productioncompany " : "Production company(s)",
                          "Released" : "Release date",
                          "Original release" : "Release date",
                          "Length" : "Running time",
                          "Country of origin" : "Country(s)",
                          "Country" : "Country(s)",
                          "Original language(s)" : "Language(s)",
                          "Language" : "Language(s)",
                          "Adaptation by" : "Writer(s)",
                          "Screen story by" : "Writer(s)",
                          "Screenplay by" : "Writer(s)",
                          "Story by" : "Writer(s)",
                          "Written by" : "Writer(s)",
                          "Voices of" : "Starring"
                          }

# func to clean movie data
def clean_movie(movie):
    movie = dict(movie) # create a non-destructive copy
    
    # remove any alternate title info and re-add it under new "Alternate title(s)" column
    alt_titles = {}
    for key in alt_title_keys:
        if key in movie:
            alt_titles[key] = movie.pop(key)
    if len(alt_titles) > 0:
        movie["Alternate title(s)"] = alt_titles
    
    # func to change column name
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    # consolidate column names
    for old_name, new_name in column_names_to_change.items():
        change_column_name(old_name, new_name)
    
    return movie

In [7]:
# clean movie and create new DataFrame
clean_movies = [clean_movie(movie) for movie in wiki_movies]
clean_movies_df = pd.DataFrame(clean_movies)
sorted(clean_movies_df.columns.tolist())

['Alternate title(s)',
 'Animator(s)',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country(s)',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language(s)',
 'Narrated by',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Writer(s)',
 'imdb_link',
 'title',
 'url',
 'year']

In [8]:
# extract IMDB IDs and drop any duplicate films, i.e. films with the same IMDB ID
clean_movies_df["IMDB ID"] = clean_movies_df["imdb_link"].str.extract(r"(tt\d{7})")
clean_movies_df.drop_duplicates(subset="IMDB ID", inplace=True)
print(len(clean_movies_df))
clean_movies_df.head()

7033


,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Created by,Preceded by,Suggested by,Alternate title(s),Recorded,Venue,Label,Color process,Animator(s),IMDB ID
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


In [9]:
# print number of null values in each column
for col in clean_movies_df.columns:
    print(col + ": " + str(clean_movies_df[col].isnull().sum()))

url: 0
year: 0
imdb_link: 0
title: 1
Based on: 4852
Starring: 182
Narrated by: 6752
Cinematography: 691
Release date: 32
Running time: 139
Budget: 2295
Box office: 1548
Director: 0
Distributor: 357
Editor(s): 548
Composer(s): 518
Producer(s): 202
Production company(s): 1678
Country(s): 236
Language(s): 86
Writer(s): 199
Genre: 6923
Original network: 6908
Executive producer(s): 6936
Production location(s): 6986
Picture format: 6969
Audio format: 6972
Followed by: 7024
Created by: 7023
Preceded by: 7023
Suggested by: 7032
Alternate title(s): 7012
Recorded: 7031
Venue: 7032
Label: 7031
Color process: 7032
Animator(s): 7031
IMDB ID: 0


In [16]:
# keep only columns whose values are at least 10% non-null
columns_to_keep = [column for column in clean_movies_df.columns
                     if clean_movies_df[column].isnull().sum() < 0.9 * len(clean_movies_df)]
clean_movies_df = clean_movies_df[columns_to_keep]
clean_movies_df.head(5)

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Budget,...,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Country(s),Language(s),Writer(s),IMDB ID
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,$20 million,...,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,United States,English,"[David Arnott, James Cappe]",tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,$6 million,...,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,United States,English,"[James Foley, Robert Redlin]",tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,$35 million,...,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]",United States,"[English, Lao]","[John Eskow, Richard Rush]",tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,$12 million,...,Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,United States,English,Woody Allen,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,$25 million,...,John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,US,English,Paul Hogan,tt0099018


In [33]:
# get list of non-null box office values
box_office = clean_movies_df["Box office"].dropna()
box_office.count()

5485

In [34]:
# find non-string values in box office column
box_office[box_office.map(lambda item: type(item) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [35]:
# join each list in box office column
box_office = box_office.apply(lambda item: " ".join(item) if type(item) == list else item)

In [39]:
# replace the first half of any hyphenated values with only a $ sign
box_office = box_office.str.replace(r"\$.*[-—–](?![a-z])", "$", regex=True)

In [40]:
# find all box office values of the form $123.4 million/billion
form_one = r"\$\s*\d+[\.,]?\d*\s*[mb]illi?on"
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3909

In [41]:
# find all box office values of the form $123,456,789
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s+[mb]illion)"
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1559

In [42]:
# cross reference the lists of matches
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)
box_office[~matches_form_one & ~matches_form_two]

600                      $5000 (US)
1070                     35,254,617
1480                     £3 million
1865                   ¥1.1 billion
2032                            N/A
2091                           $309
2665    926,423 admissions (France)
3631                            TBA
3879       CN¥3.650 million (China)
4116                     £7,385,434
4306                            $30
4561        $45.2k (only in Turkey)
5447                          £2.56
5784                       413 733$
6013                        Unknown
6369                          $111k
6370                           $588
6593                 less than $372
6843                        8 crore
Name: Box office, dtype: object

In [43]:
box_office.str.extract(f"({form_one}|{form_two})")

,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7070,$19.4 million
7071,$41.9 million
7072,$76.1 million
7073,$38.4 million


In [44]:
# parse box office values into numeric values
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan
    
    # set return value and mutiplier to NaN
    # these will only be changed if input matches a regex form
    value = np.nan
    multiplier = np.nan

    # if input is of the form $###.# million
    if re.match(r"\$\s*\d+[\.,]?\d*\s*milli?on", s, flags=re.IGNORECASE):
        
        # set multiplier to a million
        multiplier = 10**6

    # if input is of the form $###.# billion
    elif re.match(r"\$\s*\d+[\.,]?\d*\s*billi?on", s, flags=re.IGNORECASE):
        
        # set multiplier to a billion
        multiplier = 10**9

    # if input is of the form $###,###,###
    elif re.match(r"\$\s*\d{1,3}(,\d{3})+", s, flags=re.IGNORECASE):
        
        # set multiplier to 1
        multiplier = 1

    # if multiplier has changed, i.e. if input matches any regex form,
    # remove non-digit characters and convert remaining number
    if multiplier != np.nan:
        value = float(re.sub("\$|\s|,|[A-Za-z]","",s)) * multiplier

    # return value of new number, or NaN if no forms matched
    return value

In [48]:
# replace box office data in clean_movies_df with parsed numbers
clean_movies_df["Box office"] = box_office.str.extract(f"({form_one}|{form_two})", flags=re.IGNORECASE)[0].apply(parse_dollars)

In [49]:
clean_movies_df["Box office"]

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7071    41900000.0
7072    76100000.0
7073    38400000.0
7074     5500000.0
7075           NaN
Name: Box office, Length: 7033, dtype: float64

In [55]:
# get non-null budget values
budget = clean_movies_df["Budget"].dropna()

# join any list values
budget = budget.map(lambda item: " ".join(item) if type(item) == list else item)

# remove any characters between $ sign and hyphens
budget = budget.str.replace(r"\$.*[-—–](?![a-z])", "$", regex=True)

# remove citations
budget = budget.str.replace(r"\[\d\]", "", regex=True)

In [56]:
# show budget values that do not match defined forms
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

136                         Unknown
204     60 million Norwegian Kroner
478                         Unknown
1226                        Unknown
1278                            HBO
1374                     £6,000,000
1397                     13 million
1480                   £2.8 million
1734                   CAD2,000,000
1913     PHP 85 million (estimated)
1948                    102,888,900
1953                   3,500,000 DM
1973                     ₤2,300,874
2281                     $14 milion
2451                     ₤6,350,000
3144                   € 40 million
3418                        $218.32
3802                   £4.2 million
3906                            N/A
3959                    760,000 USD
4470                       19 crore
4641                    £17 million
5424                            N/A
5447                     £4 million
5671                    €14 million
5687                   $ dead link]
6385                  £ 12 million 
6593                     £3 

In [57]:
# parse budget values and replace budget column in clean_movies_df
clean_movies_df["Budget"] = budget.str.extract(f"({form_one}|{form_two})", flags=re.IGNORECASE)[0].apply(parse_dollars)

In [61]:
# make list of non-null release date values and join any lists
release_date = clean_movies_df["Release date"].dropna().apply(lambda item: " ".join(item) if type(item) == list else item)
release_date.tolist()

['July 11, 1990 ( 1990-07-11 )',
 'May 17, 1990 ( 1990-05-17 ) (Cannes Film Market) August 24, 1990 ( 1990-08-24 ) (United States)',
 'August 10, 1990 ( 1990-08-10 )',
 'December 25, 1990 ( 1990-12-25 )',
 'December 19, 1990',
 'March 22, 1990 ( 1990-03-22 )',
 'October 6, 1990 ( 1990-10-06 ) ( New York Film Festival )',
 'March 8, 1991 ( 1991-03-08 )',
 'March 7, 1990 ( 1990-03-07 )',
 'February 23, 1990 ( USA )',
 'June 8, 1990 ( 1990-06-08 )',
 'July 18, 1990 ( 1990-07-18 )',
 'October 5, 1990 ( 1990-10-05 ) (United States)',
 'December 20, 1990 (limited U.S.) January 11, 1991 (wide U.S.)',
 'May 25, 1990 ( 1990-05-25 )',
 'March 9, 1990',
 'March 2, 1990 ( 1990-03-02 )',
 'June 22, 1990 ( 1990-06-22 )',
 'May 18, 1990 ( 1990-05-18 )',
 'February 11, 1990',
 'October 12, 1990 ( 1990-10-12 )',
 'January 1990 ( 1990-01 ) ( Sundance Film Festival ) March 16, 1990 ( 1990-03-16 )',
 'December 21, 1990 ( 1990-12-21 )',
 'September 8, 1990 ( 1990-09-08 ) ( TIFF ) February 22, 1991 ( 1991-0